In [ ]:
from pathlib import Path
import pandas as pd

# Arrhythmia Dataset

- Number of Samples: 109446
- Number of Categories: 5
- Sampling Frequency: 125Hz
- Data Source: Physionet's MIT-BIH Arrhythmia Dataset
- Classes:
    - 'Normal': 0, 
    - 'Supraventricular ectopic beat': 1, 
    - 'Ventricular ectopic beat': 2, 
    - 'Fusion beat': 3, 
    - 'Unknown beat': 4
- All the samples are cropped, downsampled and padded with zeroes if necessary to the fixed dimension of 187. There is a target column named "target".

How is the train/test split?

In [ ]:
datadir = Path('/data/mads-hack-storage/')
trainfile = datadir / 'heart_big_train.parq'
testfile = datadir / 'heart_big_test.parq'
validfile = datadir / 'heart_big_valid.parq'

train_df = pd.read_parquet(trainfile)
test_df = pd.read_parquet(testfile)
valid_df = pd.read_parquet(validfile)
len(train_df), len(test_df), len(valid_df)

And what about the balance of the target?

In [ ]:
train_df.target.value_counts(), test_df.target.value_counts(), valid_df.target.value_counts()
train_percentages = train_df.target.value_counts(normalize=True) * 100
test_percentages = test_df.target.value_counts(normalize=True) * 100
valid_percentages = valid_df.target.value_counts(normalize=True) * 100
train_percentages, test_percentages, valid_percentages

What does the signal look like?

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

signal = train_df.iloc[2, :-1]
print(signal)

sns.lineplot(data=signal)
plt.xticks(range(0, len(signal), 25));

In [ ]:
train_df.columns